In [1]:
import os

import pandas as pd
import numpy as np
from scipy import stats
from scipy import io as sio
from scipy.optimize import least_squares
import matplotlib.pyplot as plt
from matplotlib import patches

In [2]:
def wrp(x):
    bigger90=x>np.pi/2
    smaller90=x<-np.pi/2
    x[bigger90]=x[bigger90]-np.pi
    x[smaller90]=x[smaller90]+np.pi
    return x
def wrpdeg(x):
    bigger90=x>90
    smaller90=x<-90
    x[bigger90]=x[bigger90]-180
    x[smaller90]=x[smaller90]+180
    return x

In [3]:
df=pd.read_csv('preposed_rt_rs.csv')
df.shape

(15544, 25)

In [4]:
# precision ACROSS subjects
rdf=df[df['debadtrial']!=True]
precisiondf=rdf[['sub','abs_angle_diff','abs_angle_diff_de']].groupby('sub').mean()
precisiondf

,abs_angle_diff,abs_angle_diff_de
sub,,
1,7.348737,2.698339
2,11.891938,5.221226
3,9.716592,5.224159
5,7.155765,3.244711
6,16.878181,8.176792
7,10.155733,6.931905
8,12.049206,5.493938
9,8.769166,5.535028
11,8.735768,4.102143


In [5]:
print("1-item trial")
print(np.mean(np.array(precisiondf['abs_angle_diff_de']))," ± ",np.std(np.array(precisiondf['abs_angle_diff_de'])))
print("3-item trial")
print(np.mean(np.array(precisiondf['abs_angle_diff']))," ± ",np.std(np.array(precisiondf['abs_angle_diff'])))

1-item trial
5.333497128680767  ±  1.367003956137672
3-item trial
10.05644403931695  ±  3.1972479917157783


In [6]:
meanbias=np.nanmean(np.array(rdf['angle_diff_de']))
print(meanbias)
rdf['angle_diff_de']=wrpdeg(rdf['angle_diff_de']-meanbias)
hdf=rdf[rdf['condition']==1]
ldf=rdf[rdf['condition']==0]

0.2969816835752013


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
# get numss
hpmidirec=np.array(hdf['pmi_ori'])
himidirec=np.array(hdf['imi_ori'])
hcdirec=np.array(hdf['c_ori'])
hdedirec=np.array(hdf['ori_to_report_de_convt'])
hdeerror=np.array(hdf['angle_diff_de'])

lpmidirec=np.array(ldf['pmi_ori'])
limidirec=np.array(ldf['imi_ori'])
lcdirec=np.array(ldf['c_ori'])
ldedirec=np.array(ldf['ori_to_report_de_convt'])
ldeerror=np.array(ldf['angle_diff_de'])


h_pmi = np.deg2rad(hpmidirec)
h_imi = np.deg2rad(himidirec)
h_c = np.deg2rad(hcdirec)
h_de = np.deg2rad(hdedirec)


l_pmi = np.deg2rad(lpmidirec)
l_imi = np.deg2rad(limidirec)
l_c = np.deg2rad(lcdirec)
l_de = np.deg2rad(ldedirec)

h_deerror = np.deg2rad(hdeerror)
l_deerror = np.deg2rad(ldeerror)

highdf=pd.DataFrame(data={'pmi':h_pmi,'imi':h_imi,'c':h_c,'de':h_de,'de_error':h_deerror})
lowdf=pd.DataFrame(data={'pmi':l_pmi,'imi':l_imi,'c':l_c,'de':l_de,'de_error':l_deerror})

In [8]:
def dog(x, a, w):
    c = np.sqrt(2) / np.exp(-0.5)
    return x * a * w * c * np.exp(-(w * x) ** 2)


def fit_dog(y, x):

    def _solver(params):
        a, w = params
        return y - dog(x, a, w)

    min_a = -np.pi/2
    max_a = np.pi/2

    min_w = 0.8
    max_w = 8.0

    min_cost = np.inf
    for _ in range(200):
        params_0 = [np.random.rand() * (max_a - min_a) + min_a,
                    np.random.rand() * (max_w - min_w) + min_w]
        try:
            result = least_squares(_solver, params_0,
                                   bounds=([min_a, min_w],
                                           [max_a, max_w]))
        except ValueError:
            continue
        if result['cost'] < min_cost:
            best_params, min_cost = result['x'], result['cost']
    try:
        return best_params[0], best_params[1], min_cost
    except UnboundLocalError:
        return np.nan, np.nan, min_cost

In [9]:
def indirect(a1, w1,a2,w2,imi,c,de):
    d_c_imi=wrp(imi-c)
    bias_c=dog(d_c_imi, a1, w1)
    c_prime=wrp(c+bias_c)
    d_de_cprime=wrp(c_prime-de)
    bias_de = dog(d_de_cprime, a2, w2)

    return bias_de

def fit_indirect(y, imi,c,de):

    def _solver(params):
        a1, w1,a2,w2 = params
        return y - indirect(a1, w1,a2,w2,imi,c,de)

    min_a = -np.pi/2
    max_a = np.pi/2

    min_w = 0.8
    max_w = 8.0

    min_cost = np.inf
    for _ in range(200):
        params_0 = [np.random.rand() * (max_a - min_a) + min_a,
                    np.random.rand() * (max_w - min_w) + min_w,
                    np.random.rand() * (max_a - min_a) + min_a,
                    np.random.rand() * (max_w - min_w) + min_w]
        try:
            result = least_squares(_solver, params_0,
                                   bounds=([min_a, min_w,min_a, min_w],[max_a, max_w,max_a, max_w]))
        except ValueError:
            continue
        if result['cost'] < min_cost:
            best_params, min_cost = result['x'], result['cost']
    try:
        return best_params, min_cost
    except UnboundLocalError:
        return np.nan, np.nan
    
def direct(a1, w1,imi,de):
    d_de_imi=wrp(imi-de)
    bias_de = dog(d_de_imi, a1, w1)

    return bias_de

def fit_direct(y, imi,de):

    def _solver(params):
        a1, w1 = params
        return y - direct(a1, w1,imi,de)

    min_a = -np.pi/2
    max_a = np.pi/2

    min_w = 0.8
    max_w = 8.0

    min_cost = np.inf
    for _ in range(200):
        params_0 = [np.random.rand() * (max_a - min_a) + min_a,
                    np.random.rand() * (max_w - min_w) + min_w]
        try:
            result = least_squares(_solver, params_0,
                                   bounds=([min_a, min_w],[max_a, max_w]))
        except ValueError:
            continue
        if result['cost'] < min_cost:
            best_params, min_cost = result['x'], result['cost']
    try:
        return best_params, min_cost
    except UnboundLocalError:
        return np.nan, np.nan
    
def fit_zero(y):

    def _solver(params):
        a1, w1 = params
        return y - 0

    min_a = -np.pi/2
    max_a = np.pi/2

    min_w = 0.8
    max_w = 8.0

    min_cost = np.inf
    for _ in range(200):
        params_0 = [np.random.rand() * (max_a - min_a) + min_a,
                    np.random.rand() * (max_w - min_w) + min_w]
        try:
            result = least_squares(_solver, params_0,
                                   bounds=([min_a, min_w],[max_a, max_w]))
        except ValueError:
            continue
        if result['cost'] < min_cost:
            best_params, min_cost = result['x'], result['cost']
    try:
        return best_params, min_cost
    except UnboundLocalError:
        return np.nan, np.nan

In [10]:
# AIC=n*log(SSR/n)+2k
# AICc=AIC+2k(k+1)/(n-k-1)

h_indirect_para,h_indirect_SSR=fit_indirect(h_deerror, h_imi,h_c,h_de)
h_indirect_SSR=h_indirect_SSR*2
n=len(h_deerror)
n_para=4
print('HIGH indirect model AICc:')
print(n*np.log(h_indirect_SSR/n)+2*n_para+2*n_para*(n_para+1)/(n-n_para-1))

h_direct_para,h_direct_SSR=fit_direct(h_deerror, h_imi,h_de)
h_direct_SSR=h_direct_SSR*2
n=len(h_deerror)
n_para=2
print('HIGH direct model AICc:')
print(n*np.log(h_direct_SSR/n)+2*n_para+2*n_para*(n_para+1)/(n-n_para-1))

h_zero_para,h_zero_SSR=fit_zero(h_deerror)
h_zero_SSR=h_zero_SSR*2
n=len(h_deerror)
n_para=0
print('HIGH null model AICc:')
print(n*np.log(h_zero_SSR/n)+2*n_para+2*n_para*(n_para+1)/(n-n_para-1))

HIGH indirect model AICc:
-31693.038077670568
HIGH direct model AICc:
-31681.366651378754
HIGH null model AICc:
-31671.17154959448


In [11]:
print(-31693.038077670568-(-31681.366651378754))
print(-31681.366651378754-(-31671.17154959448))

-11.671426291813987
-10.195101784272993


In [12]:
h_indirect_para

array([-0.31478224,  2.45174923,  0.01510491,  2.23907238])

In [13]:
# AIC=n*log(SSR/n)+2k
# AICc=AIC+2k(k+1)/(n-k-1)

l_indirect_para,l_indirect_SSR=fit_indirect(l_deerror, l_imi,l_c,l_de)
l_indirect_SSR=l_indirect_SSR*2
n=len(l_deerror)
n_para=4
print('LOW indirect model AICc:')
print(n*np.log(l_indirect_SSR/n)+2*n_para+2*n_para*(n_para+1)/(n-n_para-1))

l_direct_para,l_direct_SSR=fit_direct(l_deerror, l_imi,l_de)
l_direct_SSR=l_direct_SSR*2
n=len(l_deerror)
n_para=2
print('LOW direct model AICc:')
print(n*np.log(l_direct_SSR/n)+2*n_para+2*n_para*(n_para+1)/(n-n_para-1))

l_zero_para,l_zero_SSR=fit_zero(l_deerror)
l_zero_SSR=l_zero_SSR*2
n=len(l_deerror)
n_para=0
print('LOW null model AICc:')
print(n*np.log(l_zero_SSR/n)+2*n_para+2*n_para*(n_para+1)/(n-n_para-1))

LOW indirect model AICc:
-31502.115001564704
LOW direct model AICc:
-31477.955583974785
LOW null model AICc:
-31477.8603948671


In [14]:
print(-31502.115001564704-(-31477.955583974785))
print(-31477.955583974785-(-31477.8603948671))

-24.15941758991903
-0.09518910768383648


In [15]:
l_indirect_para

array([-1.29917299,  8.        ,  0.0155828 ,  2.1194799 ])